In [3]:
#tools
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [4]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="Query arxiv papers")
print(arxiv.name)

arxiv


In [5]:
arxiv.invoke("Attention is all you need.")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [6]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=500)
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper_wiki,description="Query wiki information")
print(wikipedia.name)

wikipedia


In [7]:
wikipedia.invoke("What's the date of birth of Virat Kohli")

"Page: Career of Virat Kohli\nSummary: Virat Kohli's senior career began when he made his debut in List A cricket, playing against Services in the Ranji One-Day Trophy, but he did not have the opportunity to bat during the match. On the international stage, he has been representing India since he was included in the ODl squad for the tour of Sri Lanka. Kohli was part of the team during India won the 2011 Cricket World Cup, the 2013 ICC Champions Trophy, the 2024 ICC T20 World Cup and 2025 ICC Cham"

In [11]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [13]:
# Tavily - It's a tool to connect LLM to internet
from langchain_tavily import TavilySearch

tavily = TavilySearch()

In [14]:
tavily.invoke("Tell me the recent news about Formula 1")

{'query': 'Tell me the recent news about Formula 1',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.newsnow.com/us/Sports/F1',
   'title': 'F1 News | Latest Formula One News - NewsNow',
   'content': 'Latest F1 news in a live feed, including breaking news, race results, driver interviews, analysis, commentary and all updates relating to Formula 1 racing.',
   'score': 0.74273956,
   'raw_content': None},
  {'url': 'https://www.motorsport.com/f1/news/',
   'title': 'The Latest Formula 1 News, Articles & F1 Results - Motorsport.com',
   'content': 'Updated F1 news and LIVE text coverage on all GP races. From practice and qualifying to the main race event. Photos, videos, results, driver stats and more.',
   'score': 0.6780931,
   'raw_content': None},
  {'url': 'https://www.formula1.com/en/latest',
   'title': 'Latest F1 News',
   'content': "Don't miss a Formula 1 moment – with the latest news, videos, standings and results. Go behind the

In [22]:
tools = [arxiv, wikipedia, tavily]

from langchain_groq import ChatGroq
ll=ChatGroq(model="llama-3.1-8b-instant")

In [23]:
ll.invoke("Tell me the 2025 IPL winner")

AIMessage(content="I don't have information on the 2025 IPL winner. My knowledge cutoff is December 2023, and the 2025 IPL season has not yet occurred. I can provide information on past IPL winners, though.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 43, 'total_tokens': 89, 'completion_time': 0.070452668, 'prompt_time': 0.003280207, 'queue_time': 0.034822795, 'total_time': 0.073732875}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_020e283281', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--22284890-24cb-417b-a8ec-135d28bb93a2-0', usage_metadata={'input_tokens': 43, 'output_tokens': 46, 'total_tokens': 89})

In [24]:
llm_with_tools=ll.bind_tools(tools=tools)

In [ ]:
response = llm_with_tools.invoke("What is the latest news about ChatGPT Atlas")

# Display structured output
print("=" * 80)
print("RESPONSE CONTENT:")
print("=" * 80)
print(response.content)
print("\n")

# Check if there are tool calls
if hasattr(response, 'tool_calls') and response.tool_calls:
    print("=" * 80)
    print(f"TOOL CALLS ({len(response.tool_calls)}):")
    print("=" * 80)
    for i, tool_call in enumerate(response.tool_calls, 1):
        print(f"\n[{i}] Tool: {tool_call.get('name', 'N/A')}")
        print(f"    ID: {tool_call.get('id', 'N/A')}")
        print(f"    Arguments: {tool_call.get('args', {})}")
else:
    print("=" * 80)
    print("TOOL CALLS: None")
    print("=" * 80)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '2a239x2ga', 'function': {'arguments': '{"include_favicon":true,"include_images":true,"query":"ChatGPT Atlas latest news","search_depth":"advanced","start_date":"2024-01-01","topic":"news"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 1905, 'total_tokens': 1964, 'completion_time': 0.085761387, 'prompt_time': 0.121531379, 'queue_time': 0.075672186, 'total_time': 0.207292766}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--88c120a6-fe44-4cb7-9d71-d20170aa8103-0', tool_calls=[{'name': 'tavily_search', 'args': {'include_favicon': True, 'include_images': True, 'query': 'ChatGPT Atlas latest news', 'search_depth': 'advanced', 'start_date': '2024-01-01', 'topic': 'news'}, 'id': '2a239x2ga', 'type': 'tool_call'}], usage_metadata={'inp

### Alternative: Using JSON for even more structured output


In [ ]:
import json

response = llm_with_tools.invoke("What is the latest news about ChatGPT Atlas")

# Create structured output dictionary
output = {
    "content": response.content,
    "tool_calls": [],
    "response_metadata": response.response_metadata if hasattr(response, 'response_metadata') else None
}

# Add tool calls if present
if hasattr(response, 'tool_calls') and response.tool_calls:
    for tool_call in response.tool_calls:
        output["tool_calls"].append({
            "name": tool_call.get('name'),
            "id": tool_call.get('id'),
            "args": tool_call.get('args', {})
        })

# Pretty print JSON
print(json.dumps(output, indent=2, ensure_ascii=False))


### Helper function for structured output


In [ ]:
def print_structured_response(response, show_metadata=False):
    """
    Print LLM response in a structured, readable format.
    """
    print("╔" + "=" * 78 + "╗")
    print("║" + " " * 20 + "LLM RESPONSE" + " " * 46 + "║")
    print("╠" + "=" * 78 + "╣")
    print("║ CONTENT:" + " " * 69 + "║")
    print("╠" + "─" * 78 + "╣")
    
    # Print content with proper indentation
    content_lines = str(response.content).split('\n')
    for line in content_lines:
        # Truncate very long lines
        if len(line) > 76:
            line = line[:73] + "..."
        print(f"║ {line:<77}║")
    
    print("╠" + "=" * 78 + "╣")
    
    # Tool calls section
    if hasattr(response, 'tool_calls') and response.tool_calls:
        print(f"║ TOOL CALLS: {len(response.tool_calls)}" + " " * (67 - len(str(len(response.tool_calls)))) + "║")
        print("╠" + "─" * 78 + "╣")
        for i, tool_call in enumerate(response.tool_calls, 1):
            tool_name = tool_call.get('name', 'N/A')
            print(f"║ [{i}] {tool_name}" + " " * (73 - len(tool_name) - len(str(i))) + "║")
            if tool_call.get('args'):
                args_str = str(tool_call['args'])
                if len(args_str) > 74:
                    args_str = args_str[:71] + "..."
                print(f"║     Args: {args_str:<68}║")
    else:
        print("║ TOOL CALLS: None" + " " * 62 + "║")
    
    # Metadata section (optional)
    if show_metadata and hasattr(response, 'response_metadata'):
        print("╠" + "=" * 78 + "╣")
        print("║ METADATA:" + " " * 68 + "║")
        print("╠" + "─" * 78 + "╣")
        metadata_str = str(response.response_metadata)
        if len(metadata_str) > 76:
            metadata_str = metadata_str[:73] + "..."
        print(f"║ {metadata_str:<77}║")
    
    print("╚" + "=" * 78 + "╝")

# Example usage
response = llm_with_tools.invoke("What is the latest news about ChatGPT Atlas")
print_structured_response(response)
